In [784]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [785]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [786]:
Query='''

DECLARE _end_dt_snpsht DATE DEFAULT '2022-11-30';
DECLARE _mnth_snpsht DATE DEFAULT '2022-11-01';

WITH
  ADC_data AS(
  SELECT
    customer_id,
    dealer_customer_id,
    BAN,
    Best_partices_1,
    Best_partices_2,
    Best_partices_3,
    Best_partices_4,
    Best_partices_5,
    Best_partices_6,
    Best_Practice_All,
    Best_Practice_All_flag,
    number_days_arming_disarming,
    Arming_Consistency,
    number_of_login_days,
    Login_Consistency,
    Segment,
    TC_Last3M_count_BroadbandCommFailure,
    TC_Last3M_count_CameraNotCommunicating,
    TC_Last3M_count_CameraNotReachable,
    
    
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package
  FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot=_mnth_snpsht
    AND dealer_name='TELUS Communications Inc.' ),
    


Telus_customers as(

select cust_bus_cust_id,pi_cntrct_end_ts as contract_end_date,pi_cntrct_start_ts as contract_start_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = _end_dt_snpsht #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
order by cust_bus_cust_id

)



,Telus_internet_customers as

(

select cust_bus_cust_id as cust_bus_cust_id,1 as Telus_Internet_customers
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = _end_dt_snpsht  #Snapshot of the last day of the month
and pi_prod_instnc_stat_cd in ('A')
and pi_prod_instnc_typ_cd ='HSIC'
and consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1

)


, Telus_SMHM_Deacts as


(


select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag  

--from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`
--from `divgpras-pr-579355.SHS.SHS_DEACTS_NOV2022`
from `divgpras-pr-579355.SHS.SHS_DEACTS_DEC2022`

)


select * from ADC_data as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join Telus_internet_customers as Telus_int
on ADC.dealer_customer_id=Telus_int.cust_bus_cust_id
left join Telus_SMHM_Deacts c
on ADC.BAN=c.BAN


'''

In [787]:
DF=extract_bq_data(bq_client, sql=Query)

In [788]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352502 entries, 0 to 352501
Data columns (total 27 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   customer_id                             352502 non-null  Int64  
 1   dealer_customer_id                      352502 non-null  object 
 2   BAN                                     352502 non-null  Int64  
 3   Best_partices_1                         352502 non-null  float64
 4   Best_partices_2                         352502 non-null  float64
 5   Best_partices_3                         352502 non-null  float64
 6   Best_partices_4                         352502 non-null  float64
 7   Best_partices_5                         352502 non-null  float64
 8   Best_partices_6                         352502 non-null  float64
 9   Best_Practice_All                       352502 non-null  float64
 10  Best_Practice_All_flag                  3525

In [789]:
DF.head()

,customer_id,dealer_customer_id,BAN,Best_partices_1,Best_partices_2,Best_partices_3,Best_partices_4,Best_partices_5,Best_partices_6,Best_Practice_All,...,TC_Last3M_count_CameraNotCommunicating,TC_Last3M_count_CameraNotReachable,Package,cust_bus_cust_id,contract_end_date,contract_start_date,cust_bus_cust_id_1,Telus_Internet_customers,BAN_1,Telus_Churn_Flag
0,5513588,105046621,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,Monitored,105046621,9999-12-31 00:00:00,None,None,<NA>,<NA>,<NA>
1,7582553,1144381,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,Monitored,1144381,9999-12-31 00:00:00,None,1144381,1,<NA>,<NA>
2,6721119,20100,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,Monitored,20100,2023-11-24 00:00:00,2020-11-24 00:00:00,20100,1,<NA>,<NA>
3,5504308,1872464,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,Monitored,1872464,2022-12-15 00:00:00,2019-12-15 00:00:00,1872464,1,<NA>,<NA>
4,6612761,2593136,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,Monitored,2593136,2024-11-29 00:00:00,2021-11-29 00:00:00,2593136,1,<NA>,<NA>


In [790]:
DF.fillna(0,inplace=True)

In [791]:
DF['Arming_Consistency'].value_counts(normalize=True)*100

0.000000      64.743462
100.000000     7.387192
3.333333       4.125367
96.666667      2.736155
6.666667       2.459561
93.333333      1.675451
10.000000      1.351198
90.000000      1.279425
13.333333      1.124816
86.666667      1.041980
16.666667      0.856733
83.333333      0.813329
20.000000      0.769641
80.000000      0.745528
76.666667      0.691344
23.333333      0.649642
26.666667      0.598294
73.333333      0.595174
70.000000      0.582408
30.000000      0.543827
66.666667      0.516025
33.333333      0.502125
63.333333      0.499855
56.666667      0.480281
36.666667      0.475742
60.000000      0.474040
50.000000      0.468650
40.000000      0.464678
46.666667      0.456451
43.333333      0.450778
53.333333      0.440849
Name: Arming_Consistency, dtype: float64

In [792]:
def Arming_category_making(row):
    
    if row['Arming_Consistency']==0:
        return 'Arming_0%'
    elif row['Arming_Consistency']>0 and row['Arming_Consistency']<=20 :
        return 'Arming_0_20%%'
    elif row['Arming_Consistency']>20 and row['Arming_Consistency']<=50 :
        return 'Arming_20_50%%'
    elif row['Arming_Consistency']>50:
        return 'Arming_50_more%'
    else:
        return "None_of_Above"

In [793]:

DF['Arming_category']=DF.apply(Arming_category_making,axis=1)

In [794]:
DF['Arming_category'].value_counts(normalize=True)*100

Arming_0%          64.743462
Arming_50_more%    19.959036
Arming_0_20%%      10.687315
Arming_20_50%%      4.610187
Name: Arming_category, dtype: float64

In [795]:
DF['Login_Consistency'].value_counts(normalize=True)*100

0.000000     28.947637
53.571429    25.043830
3.571429      6.016420
7.142857      4.415578
50.000000     4.202529
10.714286     3.625795
46.428571     3.282535
14.285714     3.150053
42.857143     2.929629
17.857143     2.890196
21.428571     2.693885
39.285714     2.654737
25.000000     2.618141
35.714286     2.533319
28.571429     2.498993
32.142857     2.477716
78.571429     0.003972
82.142857     0.003404
57.142857     0.003404
92.857143     0.002837
67.857143     0.001702
64.285714     0.001418
71.428571     0.001135
85.714286     0.000567
89.285714     0.000567
Name: Login_Consistency, dtype: float64

In [796]:
def login_category_making(row):
    
    if row['Login_Consistency']==0:
        return 'Login_0%'
    elif row['Login_Consistency']>0 and row['Login_Consistency']<=20 :
        return 'Login_0_20%%'
    elif row['Login_Consistency']>20 and row['Login_Consistency']<=50 :
        return 'Login_20_50%%'
    elif row['Login_Consistency']>50:
        return 'Login_50_more%'
    else:
        return "None_of_Above"

In [797]:
DF['Login_category']=DF.apply(login_category_making,axis=1)

In [798]:
DF['Login_category'].value_counts(normalize=True)*100

Login_0%          28.947637
Login_20_50%%     25.891484
Login_50_more%    25.062837
Login_0_20%%      20.098042
Name: Login_category, dtype: float64

In [799]:
DF['Telus_Churn_Flag'].value_counts()

0    349377
1      3125
Name: Telus_Churn_Flag, dtype: Int64

In [800]:
DF['Telus_Churn_Flag'].value_counts(normalize=True)*100

0    99.11348
1     0.88652
Name: Telus_Churn_Flag, dtype: Float64

In [801]:
# DF['Telus_Churn_Flag'].fillna(0,inplace=True)

In [802]:
DF['Telus_Churn_Flag'].value_counts(normalize=True)*100

0    99.11348
1     0.88652
Name: Telus_Churn_Flag, dtype: Float64

In [803]:
DF.head()

,customer_id,dealer_customer_id,BAN,Best_partices_1,Best_partices_2,Best_partices_3,Best_partices_4,Best_partices_5,Best_partices_6,Best_Practice_All,...,Package,cust_bus_cust_id,contract_end_date,contract_start_date,cust_bus_cust_id_1,Telus_Internet_customers,BAN_1,Telus_Churn_Flag,Arming_category,Login_category
0,5513588,105046621,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Monitored,105046621,9999-12-31 00:00:00,0,0,0,0,0,Arming_0%,Login_0%
1,7582553,1144381,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Monitored,1144381,9999-12-31 00:00:00,0,1144381,1,0,0,Arming_0%,Login_0%
2,6721119,20100,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Monitored,20100,2023-11-24 00:00:00,2020-11-24 00:00:00,20100,1,0,0,Arming_0%,Login_0%
3,5504308,1872464,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Monitored,1872464,2022-12-15 00:00:00,2019-12-15 00:00:00,1872464,1,0,0,Arming_0%,Login_0%
4,6612761,2593136,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Monitored,2593136,2024-11-29 00:00:00,2021-11-29 00:00:00,2593136,1,0,0,Arming_0%,Login_0%


In [804]:
# DF.to_csv('SHS_Churn_Analysis_OCT2022.csv',index=False)

In [805]:
DF['BAN'].value_counts()

604663242    340
601368567    270
0            200
605103191    162
604318996    144
            ... 
605034579      1
604692024      1
604040787      1
604715066      1
603579687      1
Name: BAN, Length: 338253, dtype: Int64

In [806]:
DF['customer_id'].value_counts()

13748337    27
13333221    27
13796294    27
13620647    27
13901039    27
            ..
13728656     1
11046839     1
12233542     1
10341827     1
6524022      1
Name: customer_id, Length: 341920, dtype: Int64

In [807]:
DF['customer_id'].count()

352502

In [808]:
DF['Telus_Internet_customers'].value_counts()

1    248198
0    104304
Name: Telus_Internet_customers, dtype: Int64

In [809]:
DF['Telus_Internet_customers'].value_counts(normalize=True)

1    0.704104
0    0.295896
Name: Telus_Internet_customers, dtype: Float64

In [810]:
DF['TC_Last3M_count_BroadbandCommFailure_flag']=DF['TC_Last3M_count_BroadbandCommFailure'].apply(lambda x: 1 if x>0 else 0)
DF['TC_Last3M_count_CameraNotCommunicating_flag']=DF['TC_Last3M_count_CameraNotCommunicating'].apply(lambda x: 1 if x>0 else 0)
DF['TC_Last3M_count_CameraNotReachable_flag']=DF['TC_Last3M_count_CameraNotReachable'].apply(lambda x: 1 if x>0 else 0)
DF['TC_Last3M_count_BroadbandCommFailure_morethan5_flag']=DF['TC_Last3M_count_BroadbandCommFailure'].apply(lambda x: 1 if x>5 else 0)


In [811]:
DF_monitored=DF[DF['Package']=='Monitored']

In [812]:
DF_monitored['Telus_Internet_customers'].value_counts(normalize=True)

1    0.624565
0    0.375435
Name: Telus_Internet_customers, dtype: Float64

In [813]:
DF_monitored.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213379 entries, 0 to 352501
Data columns (total 33 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   customer_id                                          213379 non-null  Int64  
 1   dealer_customer_id                                   213379 non-null  object 
 2   BAN                                                  213379 non-null  Int64  
 3   Best_partices_1                                      213379 non-null  float64
 4   Best_partices_2                                      213379 non-null  float64
 5   Best_partices_3                                      213379 non-null  float64
 6   Best_partices_4                                      213379 non-null  float64
 7   Best_partices_5                                      213379 non-null  float64
 8   Best_partices_6                                      2

In [814]:
pd.DataFrame(DF_monitored.groupby(['Arming_category','Login_category']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Arming_category,Login_category,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,Arming_0%,Login_0%,28924,26919,493,1.704467
1,Arming_0%,Login_0_20%%,16932,16509,122,0.720529
2,Arming_0%,Login_20_50%%,18638,18264,109,0.584827
3,Arming_0%,Login_50_more%,25139,24592,116,0.461434
4,Arming_0_20%%,Login_0%,5160,4686,64,1.240310
5,Arming_0_20%%,Login_0_20%%,10408,10085,69,0.662952
6,Arming_0_20%%,Login_20_50%%,11118,10807,60,0.539665
7,Arming_0_20%%,Login_50_more%,10502,10250,47,0.447534
8,Arming_20_50%%,Login_0%,2009,1823,14,0.696864
9,Arming_20_50%%,Login_0_20%%,2835,2733,16,0.564374


In [815]:
pd.DataFrame(DF_monitored.groupby(['Segment','Best_Practice_All']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
    
    # Best_partices_1= ('Best_partices_1','sum'),
    # Best_partices_2= ('Best_partices_2','sum'),
    # Best_partices_3= ('Best_partices_3','sum'),
    # Best_partices_4= ('Best_partices_4','sum'),
    # Best_partices_5= ('Best_partices_5','sum'),
    # Best_partices_6= ('Best_partices_6','sum'),
    # # Best_partices_All= ('Best_Practice_All','sum'),
    # Best_partices_All_flag= ('Best_Practice_All_flag','sum')
    # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Best_Practice_All,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,Disengaged,0.0,4216,3731,156,3.700190
1,Disengaged,1.0,2981,2612,53,1.777927
2,Disengaged,2.0,5727,5509,70,1.222280
3,Disengaged,3.0,4747,4517,62,1.306088
4,Disengaged,4.0,5648,5249,75,1.327904
5,Disengaged,5.0,4291,4036,63,1.468189
6,Disengaged,6.0,1314,1265,14,1.065449
7,Heavy_User,5.0,15463,15184,35,0.226347
8,Heavy_User,6.0,6519,6407,12,0.184077
9,Home_automation_Savvy,0.0,4273,4176,19,0.444652


In [817]:
pd.DataFrame(DF_monitored.groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Best_partices_1= ('Best_partices_1','sum'),
    Best_partices_2= ('Best_partices_2','sum'),
    Best_partices_3= ('Best_partices_3','sum'),
    Best_partices_4= ('Best_partices_4','sum'),
    Best_partices_5= ('Best_partices_5','sum'),
    Best_partices_6= ('Best_partices_6','sum'),
    # Best_partices_All= ('Best_Practice_All','sum'),
    Best_partices_All_flag= ('Best_Practice_All_flag','sum')
    # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Customer_count_1,Best_partices_1,Best_partices_2,Best_partices_3,Best_partices_4,Best_partices_5,Best_partices_6,Best_partices_All_flag
0,Disengaged,28924,26919,21299.0,17298.0,16066.0,4057.0,11677.0,10210.0,1314.0
1,Heavy_User,21982,21591,21810.0,21941.0,19927.0,9173.0,21759.0,21819.0,6519.0
2,Home_automation_Savvy,44739,43740,39119.0,38505.0,26158.0,10887.0,39138.0,39776.0,6994.0
3,Moderate_Users,85618,82993,64503.0,61768.0,42455.0,14834.0,51733.0,51680.0,7535.0
4,Old_Fashion,32116,31078,26335.0,24024.0,18016.0,6292.0,14826.0,15020.0,2543.0


In [818]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_1==1].groupby(['Segment','Best_partices_1']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_1,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,20317,276,1.295835
1,Heavy_User,1.0,21423,47,0.215497
2,Home_automation_Savvy,1.0,38273,186,0.475472
3,Moderate_Users,1.0,62766,320,0.496101
4,Old_Fashion,1.0,25615,81,0.307575


In [819]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_2==1].groupby(['Segment','Best_partices_2']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_2,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,16434,226,1.306509
1,Heavy_User,1.0,21550,47,0.214211
2,Home_automation_Savvy,1.0,37671,180,0.467472
3,Moderate_Users,1.0,60067,307,0.497021
4,Old_Fashion,1.0,23323,75,0.312188


In [820]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_3==1].groupby(['Segment','Best_partices_3']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_3,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,15049,217,1.350678
1,Heavy_User,1.0,19570,42,0.210769
2,Home_automation_Savvy,1.0,25646,102,0.389938
3,Moderate_Users,1.0,41194,206,0.485220
4,Old_Fashion,1.0,17407,50,0.277531


In [821]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_4==1].groupby(['Segment','Best_partices_4']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_4,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,3878,52,1.281735
1,Heavy_User,1.0,9017,17,0.185327
2,Home_automation_Savvy,1.0,10658,47,0.431708
3,Moderate_Users,1.0,14414,84,0.566267
4,Old_Fashion,1.0,6120,15,0.238398


In [822]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_5==1].groupby(['Segment','Best_partices_5']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_5,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,10836,166,1.421598
1,Heavy_User,1.0,21370,47,0.216003
2,Home_automation_Savvy,1.0,38273,184,0.470131
3,Moderate_Users,1.0,50241,273,0.527710
4,Old_Fashion,1.0,14342,55,0.370970


In [823]:
pd.DataFrame(DF_monitored[DF_monitored.Best_partices_6==1].groupby(['Segment','Best_partices_6']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_partices_6,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,9433,141,1.380999
1,Heavy_User,1.0,21432,47,0.215409
2,Home_automation_Savvy,1.0,38910,187,0.470133
3,Moderate_Users,1.0,50246,266,0.514706
4,Old_Fashion,1.0,14584,55,0.366178


In [824]:
pd.DataFrame(DF_monitored[DF_monitored.Best_Practice_All_flag==1].groupby(['Segment','Best_Practice_All_flag']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)).reset_index())

,Segment,Best_Practice_All_flag,Customer_count,Churn_total,Churn_rate
0,Disengaged,1.0,1265,14,1.065449
1,Heavy_User,1.0,6407,12,0.184077
2,Home_automation_Savvy,1.0,6870,23,0.328853
3,Moderate_Users,1.0,7325,44,0.583942
4,Old_Fashion,1.0,2471,4,0.157295


In [627]:
pd.DataFrame(DF_monitored.groupby(['Telus_Internet_customers']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    TC_Last3M_count_CameraNotReachable_flag=('TC_Last3M_count_CameraNotReachable_flag','sum'),
    TC_Last3M_count_CameraNotCommunicating_flag=('TC_Last3M_count_CameraNotCommunicating_flag','sum'),
    TC_Last3M_count_BroadbandCommFailure_flag=('TC_Last3M_count_BroadbandCommFailure_flag','sum'),
    TC_Last3M_count_BroadbandCommFailure_morethan5_flag=('TC_Last3M_count_BroadbandCommFailure_morethan5_flag','sum'),

    
    
    # Best_partices_1= ('Best_partices_1','sum'),
    # Best_partices_2= ('Best_partices_2','sum'),
    # Best_partices_3= ('Best_partices_3','sum'),
    # Best_partices_4= ('Best_partices_4','sum'),
    # Best_partices_5= ('Best_partices_5','sum'),
    # Best_partices_6= ('Best_partices_6','sum'),
    # # Best_partices_All= ('Best_Practice_All','sum'),
    # Best_partices_All_flag= ('Best_Practice_All_flag','sum')
    # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Telus_Internet_customers,Customer_count,Customer_count_1,TC_Last3M_count_CameraNotReachable_flag,TC_Last3M_count_CameraNotCommunicating_flag,TC_Last3M_count_BroadbandCommFailure_flag,TC_Last3M_count_BroadbandCommFailure_morethan5_flag,Churn_total,Churn_rate
0,0,78042,77920,7470,3734,7289,2862,468,0.599677
1,1,128864,128398,12311,5184,15180,4669,794,0.616153


In [629]:
pd.DataFrame(DF_monitored[DF_monitored.TC_Last3M_count_BroadbandCommFailure_flag>0].groupby(['Telus_Internet_customers','TC_Last3M_count_BroadbandCommFailure_flag']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Telus_Internet_customers,TC_Last3M_count_BroadbandCommFailure_flag,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,0,1,7289,7272,42,0.576211
1,1,1,15180,15145,92,0.606061


In [557]:
pd.DataFrame(DF_monitored[DF_monitored.TC_Last3M_count_CameraNotCommunicating_flag>0].groupby(['Telus_Internet_customers','TC_Last3M_count_CameraNotCommunicating_flag']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Telus_Internet_customers,TC_Last3M_count_CameraNotCommunicating_flag,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,0,1,3709,3708,19,0.512267
1,1,1,5231,5218,22,0.420570


In [630]:
pd.DataFrame(DF_monitored[DF_monitored.TC_Last3M_count_CameraNotReachable_flag>0].groupby(['Telus_Internet_customers','TC_Last3M_count_CameraNotReachable_flag']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Telus_Internet_customers,TC_Last3M_count_CameraNotReachable_flag,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,0,1,7470,7467,31,0.414993
1,1,1,12311,12279,62,0.503615


In [631]:
pd.DataFrame(DF_monitored[DF_monitored.TC_Last3M_count_BroadbandCommFailure_morethan5_flag>0].groupby(['Telus_Internet_customers','TC_Last3M_count_BroadbandCommFailure_morethan5_flag']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Telus_Internet_customers,TC_Last3M_count_BroadbandCommFailure_morethan5_flag,Customer_count,Customer_count_1,Churn_total,Churn_rate
0,0,1,2862,2853,20,0.698812
1,1,1,4669,4655,25,0.535447


In [560]:
# config= bigquery.job.LoadJobConfig()

# # config._properties['timePartitioning'] = {'field': 'Month_Year'}
# config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Table_BQ = 'SHS.SHS_Churn_Analysis_OCT2022'

# bq_table_instance= bq_client.load_table_from_dataframe(DF, Table_BQ,job_config=config)

ArrowTypeError: Expected bytes, got a 'int' object